In [1]:
import random
randomlist = random.sample(range(1, 11), 10)

In [2]:
randomlist

[3, 1, 6, 10, 7, 8, 5, 2, 4, 9]

In [1]:
!pip install -U spacy
#first install the library that would help us use BERT in an easy to use interface
#https://github.com/UKPLab/sentence-transformers/tree/master/sentence_transformers
!pip install -U sentence-transformers

     |████████████████████████████████| 1.1 MB 2.8 MB/s            
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for spacy: filename=spacy-3.2.1-cp39-cp39-macosx_11_0_arm64.whl size=5553896 sha256=4dff5b6939fc7e29930bf2ba17a3f5033b55f0375a59cfdbe7fde45d8dd51fd5
  Stored in directory: /Users/aimzlicious/Library/Caches/pip/wheels/f2/d3/33/635eb06dec7bbcb106db90d4cf82a2d37b91c6ab60239a4db0
Successfully built spacy
  Attempting uninstall: spacy
    Found existing installation: spacy 3.2.0
    Uninstalling spacy-3.2.0:
      Successfully uninstalled spacy-3.2.0


In [2]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from collections import Counter
from heapq import nlargest

Init Plugin
Init Graph Optimizer
Init Kernel


In [3]:
!python -m spacy download en_core_web_sm

Init Plugin
Init Graph Optimizer
Init Kernel
     |████████████████████████████████| 13.9 MB 13.6 MB/s            
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [4]:
!ls

Data                            combined_yosemite.csv
Hotel_Reviews.py                contextual_search.ipynb
MasterEdit copy.ipynb           ner_cities.ipynb
MasterEdit.ipynb                ner_update.ipynb
README.md                       othercode
Untitled-m-c02xl18cjhd4.ipynb   results.csv
Untitled.ipynb                  reviews_final-m-c02xl18cjhd4.py
__pycache__                     reviews_final.py
combined_lisbon.csv             trip_advisor_final.py
combined_lisbon2.csv            tripadvisor_data.csv
combined_paris.csv


In [5]:
## Use this to import spacy directly if using the Brain colab runtime or a custom colab runtime that includes spacy in its build.
import os
import spacy
nlp = spacy.load("en_core_web_sm")
from spacy import displacy

In [6]:
text = """Looking for a hotel in New York near Times Square with free breakfast and cheaper than $100 for 2nd June which is really kids friendly and has a swimming pool and I want to stay there for 8 days"""
doc = nlp(text)
sentence_spans = list(doc.sents)
displacy.render(doc, jupyter = True, style="ent")

In [7]:
text = """Looking for a hotel in Thiruvananthapuram"""
doc = nlp(text)
sentence_spans = list(doc.sents)
displacy.render(doc, jupyter = True, style="ent")

In [8]:
stopwords=list(STOP_WORDS)
from string import punctuation
punctuation=punctuation+ '\n'

In [9]:
import pandas as pd

from sentence_transformers import SentenceTransformer
import scipy.spatial
import pickle as pkl

embedder = SentenceTransformer('all-MiniLM-L6-v2')
#embedder = SentenceTransformer('bert-base-nli-mean-tokens')

In [10]:
df = pd.read_csv("combined_yosemite.csv")

In [11]:
df['Hotel'].value_counts()

Yosemite Valley Lodge                                60
Tamarack Lodge and Resort                            30
Yosemite View Lodge                                  30
WorldMark Bass Lake                                  30
Best Western Plus Yosemite Gateway Inn               30
Yosemite Southgate Hotel & Suites                    30
Wawona Hotel                                         30
The Ahwahnee                                         30
Curry Village                                        30
Yosemite West High Sierra Bed and Breakfast          30
Yosemite High Sierra Camps                           30
Upper Pines Campground                               30
Wawona Campground                                    30
Lower Pines Campground                               30
North Pines Campground                               30
Yosemite Hilltop Cabins                              30
Yosemite Vacation Homes                              30
Camp 4                                          

In [12]:
df['Hotel'].drop_duplicates()

0                                   Yosemite Valley Lodge
30                                    Yosemite View Lodge
60                           Rush Creek Lodge at Yosemite
90                                            Cedar Lodge
120                                         Red Bud Lodge
143                                       Lake View Lodge
173                           Double Eagle Resort and Spa
203                                Whispering Pines Motel
233                                       June Lake Motel
263                            White Chief Mountain Lodge
293                              Tenaya Lodge at Yosemite
323                                    Buck Meadows Lodge
353                               Yosemite Westgate Lodge
383                                      Narrow Gauge Inn
413                             Virginia Creek Settlement
443                   Yosemite Bug Rustic Mountain Resort
473     The Inn on Knowles Hill Bed and Breakfast Hotel #
503           

In [13]:
df_combined = df.sort_values(['Hotel']).groupby('Hotel', sort=False).review.apply(''.join).reset_index(name='all_review')
  

In [14]:
df_combined.head().T

,0,1,2,3,4
Hotel,Best Western Plus Yosemite Gateway Inn,Bridalveil Creek Campground,Buck Meadows Lodge,Camp 4,Cedar Lodge
all_review,We enjoyed our two night stay at the BWP Oakhu...,Bridalveil Creek is a tent campground in Yosem...,Honestly covered all our needs for our family ...,If you are a Backpacker in Yosemite you should...,We had a nice 2 nights stay here. It is close ...


In [15]:
import re

df_combined['all_review'] = df_combined['all_review'].apply(lambda x: re.sub('[^a-zA-z0-9\s]','',x))

def lower_case(input_str):
    input_str = input_str.lower()
    return input_str

df_combined['all_review']= df_combined['all_review'].apply(lambda x: lower_case(x))


In [16]:
df = df_combined

In [17]:
df_sentences = df_combined.set_index("all_review")

In [ ]:
df_sentences.head()

In [19]:
df_sentences = df_sentences["Hotel"].to_dict()
df_sentences_list = list(df_sentences.keys())
len(df_sentences_list)

52

In [ ]:
list(df_sentences.keys())[:5]

In [21]:
import pandas as pd
from tqdm import tqdm
from sentence_transformers import SentenceTransformer, util

In [22]:
df_sentences_list = [str(d) for d in tqdm(df_sentences_list)]

100%|██████████| 52/52 [00:00<00:00, 161438.79it/s]


In [23]:
# Corpus with example sentences
corpus = df_sentences_list
corpus_embeddings = embedder.encode(corpus,show_progress_bar=True)

Batches: 100%|██████████| 2/2 [00:01<00:00,  1.26it/s]


In [ ]:
#model = SentenceTransformer('all-MiniLM-L6-v2')
#paraphrases = util.paraphrase_mining(model, corpus)
#query_embeddings_p =  util.paraphrase_mining(model, queries,show_progress_bar=True)

In [ ]:
# import pickle as pkl
# with open("/content/drive/MyDrive/BertSentenceSimilarity/Pickles/corpus_embeddings.pkl" , "wb") as file_:
#  pkl.dump(corpus_embeddings,file_)

In [33]:
queries = ['Hotel which is pet friendly and best value',
           'Free breakfast'
           ]
query_embeddings = embedder.encode(queries,show_progress_bar=True)


Batches: 100%|██████████| 1/1 [00:00<00:00, 95.43it/s]


In [34]:

# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
closest_n = 5
print("\nTop 5 most similar sentences in corpus:")
for query, query_embedding in zip(queries, query_embeddings):
    distances = scipy.spatial.distance.cdist([query_embedding], corpus_embeddings, "cosine")[0]

    results = zip(range(len(distances)), distances)
    results = sorted(results, key=lambda x: x[1])

    print("\n\n=========================================================")
    print("==========================Query==============================")
    print("===",query,"=====")
    print("=========================================================")


    for idx, distance in results[0:closest_n]:
        print("Score:   ", "(Score: %.4f)" % (1-distance) , "\n" )
        #print("Paragraph:   ", corpus[idx].strip(), "\n" )
        row_dict = df.loc[df['all_review']== corpus[idx]]
        print("paper_id:  " , row_dict['Hotel'] , "\n")
        # print("Title:  " , row_dict["title"][corpus[idx]] , "\n")
        # print("Abstract:  " , row_dict["abstract"][corpus[idx]] , "\n")
        # print("Abstract_Summary:  " , row_dict["abstract_summary"][corpus[idx]] , "\n")
        print("-------------------------------------------")


Top 5 most similar sentences in corpus:


==========================Query==============================
=== Hotel which is pet friendly and best value =====
Score:    (Score: 0.5358) 

paper_id:   10    Groveland Hotel
Name: Hotel, dtype: object 

-------------------------------------------
Score:    (Score: 0.5350) 

paper_id:   13    Hounds Tooth Inn
Name: Hotel, dtype: object 

-------------------------------------------
Score:    (Score: 0.5193) 

paper_id:   27    Tenaya Lodge at Yosemite
Name: Hotel, dtype: object 

-------------------------------------------
Score:    (Score: 0.5145) 

paper_id:   32    Virginia Creek Settlement
Name: Hotel, dtype: object 

-------------------------------------------
Score:    (Score: 0.4609) 

paper_id:   51    Yosemite Westgate Lodge
Name: Hotel, dtype: object 

-------------------------------------------


==========================Query==============================
=== Free breakfast =====
Score:    (Score: 0.4027) 

paper_id:   0    Best 

In [27]:
from sentence_transformers import SentenceTransformer, util
import torch
embedder = SentenceTransformer('all-MiniLM-L6-v2')

corpus_embeddings = embedder.encode(corpus, convert_to_tensor=True)

In [32]:



# Query sentences:
queries = ['Hotel which is pet friendly and is best value',
           'Free breakfast'
           ]


# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
top_k = min(5, len(corpus))
for query in queries:
    query_embedding = embedder.encode(query, convert_to_tensor=True)

    # We use cosine-similarity and torch.topk to find the highest 5 scores
    cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:")

    for score, idx in zip(top_results[0], top_results[1]):
        print("(Score: {:.4f})".format(score))
        print(corpus[idx], "(Score: {:.4f})".format(score))
        row_dict = df.loc[df['all_review']== corpus[idx]]
        print("paper_id:  " , row_dict['Hotel'] , "\n")
    # for idx, distance in results[0:closest_n]:
    #     print("Score:   ", "(Score: %.4f)" % (1-distance) , "\n" )
    #     print("Paragraph:   ", corpus[idx].strip(), "\n" )
    #     row_dict = df.loc[df['all_review']== corpus[idx]]
    #     print("paper_id:  " , row_dict['Hotel'] , "\n")
    """
    # Alternatively, we can also use util.semantic_search to perform cosine similarty + topk
    hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=5)
    hits = hits[0]      #Get the hits for the first query
    for hit in hits:
        print(corpus[hit['corpus_id']], "(Score: {:.4f})".format(hit['score']))
    """






Query: Hotel which is pet friendly and is best value

Top 5 most similar sentences in corpus:
(Score: 0.5335)
cute and quaint with amazing hospitality is what makes the houndstooth a 5 star  grace at front desk was fantastic  food during breakfast was delicious  room was spacious a bit dated but very clean and nicely oversized with a jaccuzi   would stay againwhat a fantastic locale the rooms were clean perhaps recently remodeled but what makes the difference is the staff everyone was smiling said a sincere good morning and was attentive the food was great they have an evening reception and snacks and breakfast is included in your stay we will definitely be backi and my wife stayed here as a base whilst visiting yosemite national park for three  days the inn is located in a decent location outside of oakhurst which has all the necessities one could need   the check in experience was great and the chap who served was polite and nice they offered some nibbles waiting in our room whic

In [ ]:
model = SentenceTransformer('sentence-transformers/paraphrase-xlm-r-multilingual-v1')
embeddings = model.encode(corpus)
#print(embeddings)

Downloading:   0%|          | 0.00/345 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.74k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/718 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/550 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
# Query sentences:
queries = ['Hotel at least 10 minutes away from Musee d Orsay',
           'outside Gare du Nord'
           ]


# Find the closest 5 sentences of the corpus for each query sentence based on cosine similarity
top_k = min(5, len(corpus))
for query in queries:
    
    query_embedding = model.encode(query, convert_to_tensor=True)

    # We use cosine-similarity and torch.topk to find the highest 5 scores
    cos_scores = util.pytorch_cos_sim(query_embedding, embeddings)[0]
    top_results = torch.topk(cos_scores, k=top_k)

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop 5 most similar sentences in corpus:")

    for score, idx in zip(top_results[0], top_results[1]):
        print("(Score: {:.4f})".format(score))
        print(corpus[idx], "(Score: {:.4f})".format(score))
        row_dict = df.loc[df['all_review']== corpus[idx]]
        print("paper_id:  " , row_dict['Hotel'] , "\n")





Query: Hotel at least 10 minutes away from Musee d Orsay

Top 5 most similar sentences in corpus:
(Score: 0.4586)
stayed for a short city break  the hotel is a 1015 metro to the central for the eiffel tower and 5 minute taxi to notre dame  we did expect it to be quite far out but were pleasantly surprised how easy it was to get to all the usual sites so if you are looking for something a little quieter away from the centre hustle and bustle this is perfect  this hotel is a bit diy with complimentary tea and coffee and a serve yourself breakfast with fresh pastries ham pancakes and boiled eggs  perfect start to a long days sightseeing  the hotel decor is qwerky cool and classy whilst the rooms are cosey and just right for a short stay  staff were more than happy to accommodate our request of a balcony over looking the city roof tops and flowers as a surprise for my partner  thejust stayed 2 nights in this hotel and i loved its simplicity style location staff room lounge area and bre

In [ ]:
df

,Hotel,all_review
0,25hours Hotel Terminus Nord,weve spent lots of time in paris and this was ...
1,Acacias Etoile Hotel,the hotel is great for value the breakfast sel...
2,COQ Hotel Paris,stayed for a short city break the hotel is a ...
3,Campanile Paris 14 - Maine Montparnasse,room was very clean transportation is very ne...
4,Cler Hotel,we had the best stay at cler hotel the locati...
...,...,...
89,Sofitel Paris Le Faubourg,4 years ago i was the last time at sofitel le ...
90,St Christopher's Gare du Nord Paris,when arriving to the area it felt a little dan...
91,St Christopher's Inn Canal Paris,ive stayed at st christopher inn canal in pari...
92,Touring Hotel,hotel is in a great location minutes walk fro...


In [ ]:
hits = util.semantic_search(query_embedding, embeddings, top_k=5)
hits = hits[0]      #Get the hits for the first query
for hit in hits:
  print (hit)
  print("(Score: {:.4f})".format(hit['score']))
  print(corpus[hit['corpus_id']])
  row_dict = df.loc[df['all_review']== corpus[hit['corpus_id']]]
  print("paper_id:  " , row_dict['Hotel'] , "\n")

{'corpus_id': 77, 'score': 0.24323494732379913}
(Score: 0.2432)
the hotel is on a small beautiful unknown square right next to the cnam on the right bank close to grand boulevard st martin areas and super accessible from the gare du nord or montparnasse via the line 4   ive stayed here several times and always had a really nice view of the square   the staff is very welcoming and competentthe little palace hotel as its name suggests is small the rooms are quite tiny and there isnt much if any closet space but it wonderfully located and a beautiful spot if youre looking for a grand hotel this isnt for you but if you are looking for a nice little room to relax in when youre not walking the city i highly recommend this hotel i dont think it would be large enough for two people but for one it is enough room the staff was very helpful and kind and its minutes from wonderful shops and cafes i will definitely come back my personal favorite was the small terrace where i took my coffee in the m

In [ ]:
max_frequency=max(word_frequencies.values())
for word in word_frequencies.keys():
    word_frequencies[word]=word_frequencies[word]/max_frequency

NameError: ignored

In [ ]:
sentence_tokens= [sent for sent in doc.sents]
print(sentence_tokens)

In [ ]:
sentence_scores = {}
for sent in sentence_tokens:
    for word in sent:
        if word.text.lower() in word_frequencies.keys():
            if sent not in sentence_scores.keys():                            
             sentence_scores[sent]=word_frequencies[word.text.lower()]
            else:
             sentence_scores[sent]+=word_frequencies[word.text.lower()]

In [ ]:
from heapq import nlargest
select_length=int(len(sentence_tokens)*0.3)
select_length
summary=nlargest(select_length, sentence_scores,key=sentence_scores.get)
summary

In [ ]:
final_summary=[word.text for word in summary]
final_summary
summary=''.join(final_summary)
summary

In [ ]:
doc = df_combined['all_review'][0]

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model = AutoModelForSeq2SeqLM.from_pretrained("t5-base")
tokenizer = AutoTokenizer.from_pretrained("t5-base")

# T5 uses a max_length of 512 so we cut the article to 512 tokens.
inputs = tokenizer("summarize: " + doc, return_tensors="pt", max_length=512, truncation=True)
outputs = model.generate(
    inputs["input_ids"], max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True
)

print(tokenizer.decode(outputs[0]))

In [ ]:
from transformers import pipeline

# Open and read the article
#f = open("article.txt", "r", encoding="utf8")
to_tokenize = doc

# Initialize the HuggingFace summarization pipeline
summarizer = pipeline("summarization",max_length = 500)
summarized = summarizer(to_tokenize, min_length=25, max_length=50)

# Print summarized text
print(summarized)